# Dataset Creation 

we have files 
- `resultsAnnotation.tsv`, 
- `datasetAnnotation.tsv`, 
- `taskAnnotation.tsv`, 
- `paper_links.tsv`, 
- `TDM_taxonomy.tsv`, 
- `TDMs_taxonomy.tsv` 
- `paper_name_taxonomy.tsv` 

Created mostly from the file `evaluation-tables.json` from [paperswithcode](https://paperswithcode.com/about)

In [31]:
# imports
import ipdb, os

from sklearn.model_selection import train_test_split 

In [1]:
with open(f"../data/resultsAnnotation.tsv", errors='replace') as f:
    resultsAnnotation = f.read().splitlines()

with open(f"../data/datasetAnnotation.tsv", errors='replace') as f:
    datasetAnnotation = f.read().splitlines()
    
with open(f"../data/taskAnnotation.tsv", errors='replace') as f:
    taskAnnotation = f.read().splitlines()
    
with open(f"../data/TDM_taxonomy.tsv", errors='replace') as f:
    TDM_taxonomy = f.read().splitlines()
    
with open(f"../data/paper_name_taxonomy.tsv", errors='replace') as f:
    paper_name_taxonomy = f.read().splitlines()

In [2]:
resultsAnnotation[5]

'1510.05067v4.pdf\tHandwritten Digit Recognition#MNIST#PERCENTAGE ERROR#0.91$Image Classification#STL-10#Percentage correct#57.32$Image Classification#CIFAR-100#Percentage correct#48.75$Image Classification#SVHN#Percentage error#10.16$Image Classification#CIFAR-10#Percentage correct#80.98'

In [3]:
datasetAnnotation[5]

'1510.05067v4.pdf\tMNIST#STL-10#CIFAR-100#SVHN#CIFAR-10'

In [4]:
taskAnnotation[5]

'1510.05067v4.pdf\tHandwritten Digit Recognition'

In [5]:
TDM_taxonomy[9]

'Deblurring#HIDE (trained on GOPRO)#PSNR (sRGB)\t8'

In [6]:
paper_name_taxonomy[5]

'1510.05067v4.pdf\t5'

In [301]:
def create_training_data(path_to_resultsAnnotation, path_to_TDM_taxonomy, path_parsed_files,
                         output_dir, test_set_portion=0.2,
                         leaderboard_threshold=5, num_negative_instances=5, allowed_unknown=10):
    
    with open(f"{path_to_resultsAnnotation}/resultsAnnotation.tsv", errors='replace') as f:
        resultsAnnotation = f.read().splitlines()
    
    paper_TDM = {}
    for paper in resultsAnnotation:
        if len(paper.split("\t")) != 2:
            continue
            
        title, TDMSList = paper.split("\t")
        
        title = '.'.join(title.split('/')[-1].split('.')[:-1])
        paper_TDM[title] = TDMSList

    with open(f"{path_to_TDM_taxonomy}/TDM_taxonomy.tsv", errors='replace') as f:
        TDM_taxonomy = f.read().splitlines()
        
    TDM_taxonomy_dict = {}
    unknown_count = 0
    for TDMCount in TDM_taxonomy:
        if len(TDMCount.split("\t")) != 2:
            ipdb.set_trace()
            continue
        TDM, count = TDMCount.split("\t")
        count = int(count)
        if count >= leaderboard_threshold:
            TDM_taxonomy_dict[TDM] = count
#     ipdb.set_trace()
    list_parsed_pdf = os.listdir(path_parsed_files)
    if '.ipynb_checkpoints' in list_parsed_pdf:
        list_parsed_pdf.remove('.ipynb_checkpoints')

    
    # ToDo: will it be interresting to use stratified ? using the label ? 
    train_valid = train_test_split(list_parsed_pdf, test_size=10/100, shuffle=True)
    train, valid = train_valid[0], train_valid[1]
    
    if os.path.exists(f"{output_dir}/train.tsv"):
        os.remove(f"{output_dir}/train.tsv")
    
    
    for paper in train :
#         ipdb.set_trace()
        with open(f"{path_parsed_files}{paper}", errors='replace') as f:
            txt = f.read().splitlines()
        content = ' '.join(txt)
#         ipdb.set_trace()
        content = re.sub(r"[^a-zA-Z0-9?,'’‘´`%]+", ' ', content).strip()
#         content = re.sub(r"[\n]+", '', content).strip()
#         content = ' '.join("None")
        
        paper_id = '.'.join(paper.split('/')[-1].split('.')[:-1])
        
        not_seen = True
        if paper_id in paper_TDM.keys():
            cache_tdm = set()
            for contrib in paper_TDM[paper_id].split("$"):
                # keep uniq TDM 
                
                if len(contrib.split("#")) != 4:
    #                 missed += 1
#                     ipdb.set_trace()
                    continue

                task, dataset, metric, score = contrib.split("#")
        
                if (f"{task}#{dataset}#{metric}" in cache_tdm):
                    continue
                
                if f"{task}#{dataset}#{metric}" in TDM_taxonomy_dict.keys():
                    not_seen = False
                    cache_tdm.add(f"{task}#{dataset}#{metric}")
                    with open(f"{output_dir}/train.tsv", "a+", encoding="utf-8") as text_file:
                        text_file.write(f"true\t{paper_id}\t{task}#{dataset}#{metric}\t{content}\n")
                        
            if not_seen and (unknown_count <= allowed_unknown):
                with open(f"{output_dir}/train.tsv", "a+", encoding="utf-8") as text_file:
                    text_file.write(f"true\t{paper_id}\tunknown\t{content}\n")
                    
                    
            random_tdm =  list(TDM_taxonomy_dict.keys()) 
            random_tdm.sort()
            for RandTDM in random_tdm[:num_negative_instances]:
                task, dataset, metric = RandTDM.split("#")
                with open(f"{output_dir}/train.tsv", "a+", encoding="utf-8") as text_file:
                    text_file.write(f"false\t{paper_id}\t{task}#{dataset}#{metric}\t{content}\n")
        else:
            print(f"Paper {paper_id} not in the resultsAnnotation.tssv file")
                        
        

In [302]:
path_grobid_full_txt = "/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/pdf_txt/"
path_latex_source_tex = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/exp/arxiv_src/"
path_latex_source_pandoc_txt = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/exp/arxiv_src_txt/"

In [303]:
create_training_data(path_to_resultsAnnotation="../data/", \
                         path_to_TDM_taxonomy="../data/", path_parsed_files=path_grobid_full_txt,
                         output_dir="../data/",
                        leaderboard_threshold=5, num_negative_instances=10, allowed_unknown=10)

> <ipython-input-301-3e5d9fbdc62d>(55)create_training_data()
     54 
---> 55         paper_id = '.'.join(paper.split('/')[-1].split('.')[:-1])
     56 



ipdb>  l


     50         ipdb.set_trace()
     51 #         content = re.sub(r"[^a-zA-Z0-9?,'’‘´`%]+", '', content).strip()
     52 #         content = re.sub(r"[\n]+", '', content).strip()
     53 #         content = ' '.join("None")
     54 
---> 55         paper_id = '.'.join(paper.split('/')[-1].split('.')[:-1])
     56 
     57         not_seen = True
     58         if paper_id in paper_TDM.keys():
     59             cache_tdm = set()
     60             for contrib in paper_TDM[paper_id].split("$"):



ipdb>  content


'Title\tEvaluation of Deep Convolutional Nets for Document Image Classification and Retrieval Abstract:\tThis paper presents a new state-of-the-art for document image classification and retrieval, using features learned by deep convolutional neural networks (CNNs). In object and scene analysis, deep neural nets are capable of learning a hierarchical chain of abstraction from pixel inputs to concise and descriptive representations. The current work explores this capacity in the realm of document analysis, and confirms that this representation strategy is superior to a variety of popular hand-crafted alternatives. Experiments also show that (i) features extracted from CNNs are robust to compression, (ii) CNNs trained on non-document images transfer well to document analysis tasks, and (iii) enforcing region-specific feature-learning is unnecessary given sufficient training data. This work also makes available a new labelled subset of the IIT-CDIP collection, containing 400,000 document i

ipdb>  content = re.sub(r"[^a-zA-Z0-9?,'’‘´`%]+", ' ', content).strip()
ipdb>  content


"Title Evaluation of Deep Convolutional Nets for Document Image Classification and Retrieval Abstract This paper presents a new state of the art for document image classification and retrieval, using features learned by deep convolutional neural networks CNNs In object and scene analysis, deep neural nets are capable of learning a hierarchical chain of abstraction from pixel inputs to concise and descriptive representations The current work explores this capacity in the realm of document analysis, and confirms that this representation strategy is superior to a variety of popular hand crafted alternatives Experiments also show that i features extracted from CNNs are robust to compression, ii CNNs trained on non document images transfer well to document analysis tasks, and iii enforcing region specific feature learning is unnecessary given sufficient training data This work also makes available a new labelled subset of the IIT CDIP collection, containing 400,000 document images across 16

ipdb>  q


BdbQuit: 

## View created data

In [288]:
import pandas as pd

In [296]:
train = pd.read_csv(f"../data/train.tsv", 
                    sep="\t", names=["label", "title", "TDM", "Context"])

In [298]:
train.head()

,label,title,TDM,Context
0,True,1903.10176v3,Image Super-Resolution#Set5 - 8x upscaling#PSNR,TitleDeepREDDeepImagePriorPoweredbyREDAbstract...
1,True,1903.10176v3,Image Super-Resolution#Set14 - 8x upscaling#PSNR,TitleDeepREDDeepImagePriorPoweredbyREDAbstract...
2,True,1903.10176v3,Image Super-Resolution#Set5 - 4x upscaling#PSNR,TitleDeepREDDeepImagePriorPoweredbyREDAbstract...
3,True,1903.10176v3,Image Super-Resolution#Set14 - 4x upscaling#PSNR,TitleDeepREDDeepImagePriorPoweredbyREDAbstract...
4,False,1903.10176v3,3D Absolute Human Pose Estimation#Human3.6M#MRPE,TitleDeepREDDeepImagePriorPoweredbyREDAbstract...


In [291]:
# train.tail()

label  \
False 1903.11816v1 3D Face Reconstruction#AFLW2000-3D#Mean NME        Title FastFCN: Rethinking Dilated Convolution in the ... Modern approaches for semantic segmentation usu... Semantic segmentation [23, 40, 4] is one of the... In this section, we first give an overview on m... FCNs [22] have achieved huge success in semanti... In our method, we propose a module to upsample ... In this section, we first introduce the most po... To exploit Deep CNNs in semantic segmentation, ... To obtain a high-resolution final feature map, ... The proposed JPU is designed for generating a f... Joint Upsampling Given a low-resolution target ... The differences between the backbone of our met... Equation 4 is an optimization problem, which ta...  In this section, we first introduce the datase...   
                   3D Face Reconstruction#AFLW2000-3D#NME             Title FastFCN: Rethinking Dilated Convolution in the ... Modern approaches for semantic segmentation usu... Semantic segmentation [23, 40, 4] is one of the... In this section, we first give an overview on m... FCNs [22] have achieved huge success in semanti... In our method, we propose a module to upsample ... In this section, we first introduce the most po... To exploit Deep CNNs in semantic segmentation, ... To obtain a high-resolution final feature map, ... The proposed JPU is designed for generating a f... Joint Upsampling Given a low-resolution target ... The differences between the backbone of our met... Equation 4 is an optimization problem, which ta...  In this section, we first introduce the datase...   
                   3D Face Reconstruction#Florence#Average 3D Error   Title FastFCN: Rethinking Dilated Convolution in the ... Modern approaches for semantic segmentation usu... Semantic segmentation [23, 40, 4] is one of the... In this section, we first give an overview on m... FCNs [22] have achieved huge success in semanti... In our method, we propose a module to upsample ... In this section, we first introduce the most po... To exploit Deep CNNs in semantic segmentation, ... To obtain a high-resolution final feature map, ... The proposed JPU is designed for generating a f... Joint Upsampling Given a low-resolution target ... The differences between the backbone of our met... Equation 4 is an optimization problem, which ta...  In this section, we first introduce the datase...   
                   3D Face Reconstruction#Florence#Mean NME           Title FastFCN: Rethinking Dilated Convolution in the ... Modern approaches for semantic segmentation usu... Semantic segmentation [23, 40, 4] is one of the... In this section, we first give an overview on m... FCNs [22] have achieved huge success in semanti... In our method, we propose a module to upsample ... In this section, we first introduce the most po... To exploit Deep CNNs in semantic segmentation, ... To obtain a high-resolution final feature map, ... The proposed JPU is designed for generating a f... Joint Upsampling Given a low-resolution target ... The differences between the backbone of our met... Equation 4 is an optimization problem, which ta...  In this section, we first introduce the datase...   
                   3D Face Reconstruction#NoW Benchmark#Mean Recon... Title FastFCN: Rethinking Dilated Convolution in the ... Modern approaches for semantic segmentation usu... Semantic segmentation [23, 40, 4] is one of the... In this section, we first give an overview on m... FCNs [22] have achieved huge success in semanti... In our method, we propose a module to upsample ... In this section, we first introduce the most po... To exploit Deep CNNs in semantic segmentation, ... To obtain a high-resolution final feature map, ... The proposed JPU is designed for generating a f... Joint Upsampling Given a low-resolution target ... The differences between the backbone of our met... Equation 4 is an optimization problem, which ta...  In this section, we first introduce the datase...   

                         

In [261]:
train[train.label==True].head(10)

,label,title,TDM,Context
0,True,1607.07155v1,Face Detection#WIDER Face (Hard)#AP,N o n e
1,True,1607.07155v1,Pedestrian Detection#Caltech#Reasonable Miss Rate,N o n e
12,True,1911.01616v4,Aspect Sentiment Triplet Extraction#SemEval#F1,N o n e
23,True,1412.7259v3,Image Classification#MNIST#Percentage error,N o n e
24,True,1412.7259v3,Image Classification#STL-10#Percentage correct,N o n e
35,True,1704.07156v1,Part-Of-Speech Tagging#Penn Treebank#Accuracy,N o n e
36,True,1704.07156v1,Grammatical Error Detection#FCE#F0.5,N o n e
37,True,1704.07156v1,Grammatical Error Detection#CoNLL-2014 A2#F0.5,N o n e
38,True,1704.07156v1,Grammatical Error Detection#CoNLL-2014 A1#F0.5,N o n e
49,True,2003.12060v1,Few-Shot Image Classification#CUB 200 5-way 1-...,N o n e


In [262]:
train[train.title=="1911.08251v2"].head()

,label,title,TDM,Context
49978,True,1911.08251v2,Image Classification#STL-10#Percentage correct,N o n e
49979,False,1911.08251v2,3D Absolute Human Pose Estimation#Human3.6M#MRPE,N o n e
49980,False,1911.08251v2,3D Action Recognition#100 sleep nights of 8 ca...,N o n e
49981,False,1911.08251v2,3D Canonical Hand Pose Estimation#Ego3DHands#AUC,N o n e
49982,False,1911.08251v2,3D Canonical Hand Pose Estimation#RHP#AUC,N o n e


In [263]:
train[train.TDM=="unknown"].head()

,label,title,TDM,Context
103,True,2005.03059v3,unknown,N o n e
235,True,2102.02717v3,unknown,N o n e
276,True,2102.05126v1,unknown,N o n e
299,True,2008.06223v2,unknown,N o n e
346,True,2003.06520v1,unknown,N o n e


In [ ]:
train["len"]train.Context.apply(lambda content: len(content),)

In [171]:
tdm = pd.read_csv(f"/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/new/60Neg800unk/twofoldwithunk/fold1/train.tsv", 
                    sep="\t", names=["label", "title", "TDM", "Context"])

In [172]:
tdm.head()

,label,title,TDM,Context
0,True,1810.02575v1.pdf,Semantic Segmentation; Nighttime Driving; mIoU,Dark Model Adaptation: Semantic Image Segmenta...
1,False,1810.02575v1.pdf,Extractive Text Summarization; DebateSum; ROUGE-L,Dark Model Adaptation: Semantic Image Segmenta...
2,False,1810.02575v1.pdf,Action Recognition; Something-Something V1; To...,Dark Model Adaptation: Semantic Image Segmenta...
3,False,1810.02575v1.pdf,Multi-Object Tracking; MOTS20; sMOTSA,Dark Model Adaptation: Semantic Image Segmenta...
4,False,1810.02575v1.pdf,Continuous Control; PyBullet Ant; Return,Dark Model Adaptation: Semantic Image Segmenta...


In [173]:
tdm.tail()

,label,title,TDM,Context
256003,False,1307.0414v1.pdf,Skeleton Based Action Recognition; SHREC 2017 ...,Challenges in Representation Learning: A repor...
256004,False,1307.0414v1.pdf,Fake News Detection; FNC-1; Weighted Accuracy,Challenges in Representation Learning: A repor...
256005,False,1307.0414v1.pdf,Multimodal Unsupervised Image-To-Image Transla...,Challenges in Representation Learning: A repor...
256006,False,1307.0414v1.pdf,Graph Classification; PTC; Accuracy,Challenges in Representation Learning: A repor...
256007,False,1307.0414v1.pdf,Pose Estimation; UPenn Action; Mean PCK@0.2,Challenges in Representation Learning: A repor...


In [242]:
tdm[tdm.title =="1908.05786v1.pdf" ].head(1)

,label,title,TDM,Context
610,True,1908.05786v1.pdf,unknow,TASED-Net: Temporally-Aggregating Spatial Enco...


In [148]:
tdm[tdm.count_tdm>800].head()

,tdm,count_tdm
703,Skeleton Based Action Recognition#NTU RGB+D#Ac...,896
704,Skeleton Based Action Recognition#NTU RGB+D#Ac...,912


In [121]:
import re
# re.sub(r"[^a-zA-Z0-9?,'’‘´`%]+", '', "Hell\no*%& \n").strip()
f = re.sub(r"[\n]+", '', "Hell\no*%& \n").strip()

In [123]:
f

'Hello*%&'